In [209]:
from openai import OpenAI
import os
from dotenv import find_dotenv, load_dotenv
import json
import requests
import helpers
import config

In [210]:
token = helpers.get_token(task_name='inprompt')
task = helpers.get_task(token)

In [211]:
task['msg']

'remember information about each person and then answer the question in POLISH'

In [212]:
question = task['question']
question

'co lubi jeść na śniadanie Alojzy?'

In [213]:
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

client = OpenAI()

In [214]:
def get_name_from_question(question):
    aitext = client.completions.create(
        model="gpt-3.5-turbo-instruct",
        prompt=f"""
        Which word of provided sentence is a name of a person?
        Your answer must contain only one word - the name of a person.
        sentence: {question}
        """,
        max_tokens=50,
        temperature=0.1
    )
    answer = aitext.choices[0].text
    answer = answer.strip()
    return answer

In [215]:
name = get_name_from_question(question)
name

'Alojzy'

In [216]:
def filter_input_by_name(input, name):
    task_input = pd.Series(task['input'])
    filter_mask = task_input.str.contains(name)

    filtered_input = task_input[filter_mask]
    return filtered_input

In [217]:
filtered_input = filter_input_by_name(task['input'], name)
filtered_input

22    Alojzy ma czarne oczy, krótkie włosy i pracuje...
dtype: object

In [218]:
def get_answer_for_question(question, filtered_input):
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system", 
                "content": f"""```context: "{filtered_input.to_list()}"```
                    Odpowiedz na pytanie na podstawie powyższego kontekstu."""
             },
            {"role": "user", "content": f"{question}"}
        ],
        temperature=1
    )

    answer = completion.choices[0].message.content
    return answer

In [219]:
print(f"""question: {question} \n 
answer: {answer} \n 
context: {filtered_input.to_list()}""" )

question: co lubi jeść na śniadanie Alojzy? 
 
answer: Ernest jest fryzjerem. 
 
context: ['Alojzy ma czarne oczy, krótkie włosy i pracuje jako prawnik, a na śniadanie najbardziej lubi jeść owsiankę']


In [220]:
answer = get_answer_for_question(question, filtered_input)
answer

'Alojzy najbardziej lubi jeść owsiankę na śniadanie.'

In [221]:
helpers.send_answer(token, answer)

{'code': 0, 'msg': 'OK', 'note': 'CORRECT'}